In [1]:
from torch.optim.lr_scheduler import ReduceLROnPlateau, CyclicLR
from load_utils import prepare_data
from train_utils import *
from seq2seq_model import *
import numpy as np
import torch
from tqdm.notebook import tnrange, tqdm_notebook
from Calculate_BLEU import *
import datetime

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

In [3]:
q, a, pairs, vector = prepare_data('train', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading train -------
Read 45544 sentence pairs
Counting words
Counted words:
In questions: 16871 words
In answers: 14636 words


In [4]:
# Add EOS at end of each target sentence
pairs = [[line[0], line[1]+' EOS'] for line in pairs ]

In [5]:
matrix_len = q.n_words
weights_matrix = np.zeros((matrix_len, 300))
word_found = 0
for i, word in enumerate(q.word2index):
    try:
        weights_matrix[i] = vector[word]
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))

In [6]:
criterion = nn.NLLLoss()
optimizer = 'Adam'
batch_size = 32
lr = 0.0005

In [7]:
model = Seq2Seq(batch_size, q.n_words, a.n_words, 300, 100, weights_matrix, 0.2, device, criterion)

C:\Users\maren\Anaconda3\envs\deeplearning\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [8]:
if optimizer == 'Adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, eps=1e-6)
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [9]:
#scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, threshold=0.001)
#scheduler = CyclicLR(optimizer, base_lr=0.001, max_lr=0.1, step_size_up=500)

In [10]:
a.n_words

14636

In [11]:
from jupyterplot import ProgressPlot

pp = ProgressPlot(x_label='Epoch', plot_names=['train_loss', 'train_accuracy'])

for epoch in tnrange(1000, desc="Total epochs: "):

    print(f'Epoch {epoch + 1}: {datetime.datetime.now()}')

    # Calculte loss
    loss = epoch_train(model, optimizer, batch_size, pairs, q, a, device)
    
    #current_lr = optimizer.param_groups[0]['lr']
    
    print(f'Loss: {loss}')
    #print(f'LR: {current_lr}')
    
    #scheduler.step(loss)
    #scheduler.step()

    # Calculate accuracy
    accuracy = epoch_accuray(model, batch_size, pairs, q, a, device)

    print(f'Train accuracy: {accuracy}')

    # Calculate BLEU Score
    #BLEU_model = CalculateBleu(model, batch_size, pairs, q, a, device)
    #bleu_score = BLEU_model.score()

    #print(f'BLUE score: {bleu_score}')

    # Try to do interactive plot
    pp.update([[loss], [accuracy]])

pp.finalize()
print(f"Optimization ended successfully")


<IPython.core.display.Javascript object>

Total epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: 2021-09-16 23:05:24.259840
Loss: 2.1431635766000885
Train accuracy: 0.11720551908828875
Epoch 2: 2021-09-16 23:08:09.931197
Loss: 1.8901270065215707
Train accuracy: 0.11357689219710564
Epoch 3: 2021-09-16 23:10:55.608904
Loss: 1.8357733785828712
Train accuracy: 0.1136549795468424
Epoch 4: 2021-09-16 23:13:40.533278
Loss: 1.8373766521087804
Train accuracy: 0.11862341214557574
Epoch 5: 2021-09-16 23:16:25.439565
Loss: 1.78463294579032
Train accuracy: 0.1351137291034506
Epoch 6: 2021-09-16 23:19:09.954062
Loss: 1.8069510747478954
Train accuracy: 0.13713454131507663
Epoch 7: 2021-09-16 23:21:53.351017
Loss: 1.7443149727246399
Train accuracy: 0.1550960283906974
Epoch 8: 2021-09-16 23:24:37.654150
Loss: 1.7584241323460137
Train accuracy: 0.160082673320766
Epoch 9: 2021-09-16 23:27:22.118610
Loss: 1.7515142696587718
Train accuracy: 0.16038541645563859
Epoch 10: 2021-09-16 23:30:05.895915
Loss: 1.73190392190119
Train accuracy: 0.15123182672707963
Epoch 11: 2021-09-16 23:32:49.873033
L

Loss: 1.4284125455718826
Train accuracy: 0.22268257780286374
Epoch 86: 2021-09-17 02:58:21.408411
Loss: 1.4236733406340745
Train accuracy: 0.2207800130886098
Epoch 87: 2021-09-17 03:01:06.144979
Loss: 1.4417621690741254
Train accuracy: 0.22185118946977386
Epoch 88: 2021-09-17 03:03:50.179385
Loss: 1.4111568975607165
Train accuracy: 0.2207940991754981
Epoch 89: 2021-09-17 03:06:34.831757
Loss: 1.4475842103569136
Train accuracy: 0.2191870141332648
Epoch 90: 2021-09-17 03:09:19.269908
Loss: 1.4154260353256303
Train accuracy: 0.22550224835289961
Epoch 91: 2021-09-17 03:12:03.305932
Loss: 1.4165415084514466
Train accuracy: 0.2266509927556472
Epoch 92: 2021-09-17 03:14:47.604963
Loss: 1.4077321065260249
Train accuracy: 0.22215567243122153
Epoch 93: 2021-09-17 03:17:32.013455
Loss: 1.4415055681983258
Train accuracy: 0.22661816846840424
Epoch 94: 2021-09-17 03:20:16.082464
Loss: 1.4248789732323792
Train accuracy: 0.22397133755413648
Epoch 95: 2021-09-17 03:23:00.197033
Loss: 1.4173606800918033

Epoch 169: 2021-09-17 06:46:02.484718
Loss: 1.3389501878130554
Train accuracy: 0.2536195598170603
Epoch 170: 2021-09-17 06:48:47.196294
Loss: 1.301238875728221
Train accuracy: 0.2574138316331964
Epoch 171: 2021-09-17 06:51:32.205123
Loss: 1.3429600281330338
Train accuracy: 0.2533010438068924
Epoch 172: 2021-09-17 06:54:17.894029
Loss: 1.3291707788635183
Train accuracy: 0.2546807149600019
Epoch 173: 2021-09-17 06:57:02.292628
Loss: 1.3269461124352773
Train accuracy: 0.2577930362271175
Epoch 174: 2021-09-17 06:59:47.145024
Loss: 1.3346574339490829
Train accuracy: 0.25602575468952504
Epoch 175: 2021-09-17 07:02:31.443478
Loss: 1.3206075757855504
Train accuracy: 0.25849256409062416
Epoch 176: 2021-09-17 07:05:16.533783
Loss: 1.3251178406371638
Train accuracy: 0.25889848834565127
Epoch 177: 2021-09-17 07:08:01.318913
Loss: 1.3314906878586228
Train accuracy: 0.2558457852087146
Epoch 178: 2021-09-17 07:10:45.803023
Loss: 1.35061001889228
Train accuracy: 0.2558285321766449
Epoch 179: 2021-09-1

...

Train accuracy: 0.29516444383786233
Epoch 420: 2021-09-17 18:13:55.841714
Loss: 1.227032827389611
Train accuracy: 0.2945291821471644
Epoch 421: 2021-09-17 18:16:39.493577
Loss: 1.21594276914118
Train accuracy: 0.29947971590186756
Epoch 422: 2021-09-17 18:19:24.348177
Loss: 1.2167098929265325
Train accuracy: 0.2952476164897657
Epoch 423: 2021-09-17 18:22:08.609696
Loss: 1.2283754314493585
Train accuracy: 0.2962482994288507
Epoch 424: 2021-09-17 18:24:53.287569
Loss: 1.2148978718607981
Train accuracy: 0.299889054543316
Epoch 425: 2021-09-17 18:27:38.079067
Loss: 1.2175614398629009
Train accuracy: 0.30363986496288903
Epoch 426: 2021-09-17 18:30:22.868637
Loss: 1.2221886935047108
Train accuracy: 0.2958842934893619
Epoch 427: 2021-09-17 18:33:07.641813
Loss: 1.215231404285537
Train accuracy: 0.3021348964098955
Epoch 428: 2021-09-17 18:35:53.025699
Loss: 1.2071991372044537
Train accuracy: 0.3006940219530876
Epoch 429: 2021-09-17 18:38:39.021732
Loss: 1.212239026821535
Train accuracy: 0.29320

Loss: 1.22112578678075
Train accuracy: 0.30891467247449017
Epoch 504: 2021-09-17 22:08:42.212901
Loss: 1.210543949769554
Train accuracy: 0.30160396888924396
Epoch 505: 2021-09-17 22:11:26.738819
Loss: 1.2155775131602296
Train accuracy: 0.3012486845992623
Epoch 506: 2021-09-17 22:14:12.134324
Loss: 1.2138506559922424
Train accuracy: 0.3064788273417292
Epoch 507: 2021-09-17 22:16:57.302124
Loss: 1.1881443196057349
Train accuracy: 0.31125324528777765
Epoch 508: 2021-09-17 22:19:42.069230
Loss: 1.1955653625893423
Train accuracy: 0.3029159335026073
Epoch 509: 2021-09-17 22:22:27.663493
Loss: 1.2131586593367432
Train accuracy: 0.30032886562942884
Epoch 510: 2021-09-17 22:25:12.560519
Loss: 1.2109926047060926
Train accuracy: 0.3063287811198951
Epoch 511: 2021-09-17 22:27:57.143562
Loss: 1.2019234852456178
Train accuracy: 0.3055186256489796
Epoch 512: 2021-09-17 22:30:42.435049
Loss: 1.2145749139952735
Train accuracy: 0.30208375725035086
Epoch 513: 2021-09-17 22:33:26.869196
Loss: 1.2004451201

Epoch 587: 2021-09-18 01:56:45.242237
Loss: 1.1932803123585929
Train accuracy: 0.30803000231480576
Epoch 588: 2021-09-18 01:59:30.685729
Loss: 1.1888203043753316
Train accuracy: 0.30695349656425364
Epoch 589: 2021-09-18 02:02:15.504117
Loss: 1.1948146807032216
Train accuracy: 0.31220856027816296
Epoch 590: 2021-09-18 02:05:00.344251
Loss: 1.20947289125035
Train accuracy: 0.31334749907886383
Epoch 591: 2021-09-18 02:07:45.099802
Loss: 1.1830505120440027
Train accuracy: 0.3068757442618732
Epoch 592: 2021-09-18 02:10:30.223784
Loss: 1.1887415813247284
Train accuracy: 0.31185790270762337
Epoch 593: 2021-09-18 02:13:15.333708
Loss: 1.19499007995416
Train accuracy: 0.31330807900567426
Epoch 594: 2021-09-18 02:16:00.198794
Loss: 1.2006164365825822
Train accuracy: 0.3090418201907315
Epoch 595: 2021-09-18 02:18:45.193378
Loss: 1.181874237765285
Train accuracy: 0.30960737466077465
Epoch 596: 2021-09-18 02:21:30.567595
Loss: 1.196560129071731
Train accuracy: 0.3109089090192639
Epoch 597: 2021-09-

Train accuracy: 0.31649691739529656
Epoch 671: 2021-09-18 05:48:12.270290
Loss: 1.1930173465248035
Train accuracy: 0.31500909832181156
Epoch 672: 2021-09-18 05:50:57.447782
Loss: 1.1881589219510815
Train accuracy: 0.31953445532320024
Epoch 673: 2021-09-18 05:53:42.506009
Loss: 1.2204711360397673
Train accuracy: 0.31334294611595515
Epoch 674: 2021-09-18 05:56:27.679443
Loss: 1.1705624433907087
Train accuracy: 0.3109048748662867
Epoch 675: 2021-09-18 05:59:13.422535
Loss: 1.1753829726382785
Train accuracy: 0.31700413825187557
Epoch 676: 2021-09-18 06:01:58.824717
Loss: 1.182817006779637
Train accuracy: 0.3132857648942192
Epoch 677: 2021-09-18 06:04:43.562782
Loss: 1.1922820914120236
Train accuracy: 0.3156713322947201
Epoch 678: 2021-09-18 06:07:28.567819
Loss: 1.1862777535781024
Train accuracy: 0.31139657760819045
Epoch 679: 2021-09-18 06:10:14.097337
Loss: 1.1956250320913873
Train accuracy: 0.3153691384733159
Epoch 680: 2021-09-18 06:12:59.365783
Loss: 1.2039749435378646
Train accuracy:

KeyboardInterrupt: 

In [12]:
torch.cuda.device_count()

1